# Tối ưu tham số mô hình Neural Network

## 1. Xây dựng mô hình
- Ở video 004, chúng ta đã có code xây dựng mô hình neural network


In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.layer1 = nn.Linear(28*28, 512)
        self.hidden1 = nn.ReLU()
        self.layer2 = nn.Linear(512, 512)
        self.hidden2 = nn.ReLU()
        self.output_layer = nn.Linear(512, 10)
    
    def forward(self, x):
        x = self.flatten(x)
        x = self.layer1(x)
        x = self.hidden1(x)
        x = self.layer2(x)
        x = self.hidden2(x)
        out = self.output_layer(x)
        return out

model = NeuralNetwork()

## 2. Hyperparameters

 - **Number of Epochs** - số lần lặp qua bộ dữ liệu
 - **Batch Size** - số lượng quan sát dùng để tính đạo hàm cho mỗi lần cập nhật tham số
 - **Learning Rate** - độ lớn bước nhảy của Gradient Descent. **Lưu ý**: giá trị quá nhỏ làm tốc độ hội tụ chậm, giá trị quá lớn có thể làm nhảy quá điểm hội tụ.




In [ ]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

## 3. Hàm khoảng cách (hàm mất mát/loss function)

- Đo độ khác biệt giữa kết quả dự đoán và giá trị mục tiêu --> Cần cực tiểu hàm này
- Phổ biến:
    - Mean Square Error cho bài toán hồi quy (regression)
    - CrossEntropyLoss cho bài toán phân lớp (classification)


- Với bài toán phân lớp FashionMNIST, chúng ta dùng CrossEntropyLoss.




In [ ]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

## 4. Optimizer


In [20]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

## 5. Huấn luyện mô hình (tối ưu tham số)

- Tại mỗi bước lặp:
     * Gọi ``optimizer.zero_grad()``.
     * Gọi ``loss.backwards()`` để tính đạo hàm của loss function theo tham số.
     * Gọi ``optimizer.step()`` để cập nhật tham số.

### 5.1. Cài đặt 
- train_loop: tối lưu tham số mô hình
- test_loop: đánh giá hiệu quả mô hình trên tập test

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    n_samples = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{n_samples:>5d}]")


def test_loop(dataloader, model, loss_fn):
    n_samples = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= n_samples
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

- Vòng lặp chính:

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 15
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")